Repovizz2 DataPack Upload Example
=================================

This notebook demonstrates basic datapack operations (upload and modify)

We will use here the repovizz2 client library to ease the authentication, but it is basically a standard OAuth2 authentication.

In [1]:
from repovizz2 import RepoVizzClient

We will be using those modules:

In [2]:
import json
# This is for the notebook
from IPython.display import IFrame, display

Now, some global parameters to connect to repovizz2

In [3]:
CLIENT_ID = "4991097c-9a0c-4a56-a337-2b2177c5aa74"
CLIENT_SECRET = "97c055e7-5dd1-49e8-a6da-126d58f07ce1"


Repovizz2 authentication
-----------------------

We authenticate the client (this script) against repoVizz using a registered user. For more details on the authentication system see [repoVizz authentication]().


This will create an iframe in the notebook to authenticate the client, but you can open a new browser to do so, if you want

In [5]:
repovizz_client = RepoVizzClient(client_id=CLIENT_ID,client_secret=CLIENT_SECRET, repovizz_url="http://localhost:8080")
authorization_url = repovizz_client.start_auth()
repovizz_client.start_server(async=True)

# print authorization_url
# - or -
# import webbrowser
# webbrowser.open(authorization_url)
display(IFrame(authorization_url,800,600))

In [6]:
repovizz_client.finish_auth()
#TODO: retrieve username from repovizz_client
username = 'admin'

We will upload a new datapack

In [7]:
repovizz_client.get("/api/v1.0/users/"+username)

{u'_links': {u'collection': u'/api/v1.0/users',
  u'self': u'/api/v1.0/users/admin'},
 u'datapacks': [],
 u'email': u'admin@repovizz.upf.edu',
 u'folders': [],
 u'groups': [],
 u'username': u'admin'}

In [11]:
datapack_json = json.load(open('MYO_example_datapack/MYO_example_datapack.json'))
repovizz_client.post(
    "/api/v1.0/datapacks",
    json={
        'structure': datapack_json,
        'name': datapack_json['info']['name'],
        'owner': username,
    },
    raw=True
)


<Response [500]>

Now we upload all the assets

In [9]:
datapack_json = json.load(open('MYO_example_datapack/MYO_example_datapack.json'))

In [10]:
datapack_json

{u'children': [{u'class': u'data',
   u'link': u'video.mp4',
   u'meta': {u'mime': u'video/mp4; charset=binary'},
   u'name': u'Video',
   u'text': u'Handheld camera',
   u'type': u'e44ab9c4-a1b5-4474-b806-ece64555ef2b'},
  {u'class': u'data',
   u'link': u'audio.wav',
   u'meta': {u'mime': u'audio/x-wav; charset=binary'},
   u'name': u'Audio',
   u'text': u'Camera microphone',
   u'type': u'48e3b5d2-c34c-459c-9a2d-0756c20d80de'},
  {u'children': [{u'class': u'data',
     u'link': u'emg.csv',
     u'name': u'EMG',
     u'text': u'8-channel EMG data from the MYO armband',
     u'type': u'0746bb48-9b92-4cd3-8d9e-3e1de13269eb'}],
   u'class': u'container',
   u'name': u'EMG',
   u'text': u''},
  {u'children': [{u'class': u'data',
     u'link': u'accelerometer.csv',
     u'name': u'Accelerometer',
     u'text': u'3-dimensional (XYZ) acceleration data',
     u'type': u'ae13fb70-fa1e-47d8-9cd6-00cef4fe21cc'},
    {u'class': u'data',
     u'link': u'gyroscope.csv',
     u'name': u'Gyroscope',

In [ ]:
print authorization_url